In [157]:
import pandas as pd
import apimoex
import requests
import numpy as np
import datetime
from math import pi
from bokeh.plotting import figure, show
import time
pd.options.mode.chained_assignment = None
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [158]:
#выписываем переменные из задания
money = 100000
start = "2018-01-01"
finish = "2018-12-31"
stop_loss = 0.05
short_wind = 30
long_wind = 90

In [159]:
share = 'NVTK'
#по приколу новатек, мой робот на нем хоть какую-то прибыль генерирует

In [160]:
# с помощью session выгружаем данные с мосбиржи с интервалом, устанавливаем индексы и чистим лишнюю дату
start = datetime.datetime(2018, 1, 1)
end = datetime.datetime(2018, 12, 31)
with requests.Session() as session:
    data = apimoex.get_market_candles(session, share, interval=60, start=start, end=end)
    data = pd.DataFrame(data)
    data.set_index(pd.to_datetime(data['begin']), inplace=True)
    data.drop(['begin'], axis=1, inplace=True)
    print(data.head(), '\n')
    data.info()

                      open  close   high    low      value  volume
begin                                                             
2018-01-03 09:00:00  679.8  679.8  679.8  679.8     101970     150
2018-01-03 10:00:00  679.3  682.3  684.9  678.3   31650724   46430
2018-01-03 11:00:00  683.7  690.9  696.0  683.4  180360601  260720
2018-01-03 12:00:00  692.3  693.4  695.0  690.0   22444379   32390
2018-01-03 13:00:00  694.5  692.1  694.5  690.6   14326496   20710 

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2535 entries, 2018-01-03 09:00:00 to 2018-12-29 18:00:00
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    2535 non-null   float64
 1   close   2535 non-null   float64
 2   high    2535 non-null   float64
 3   low     2535 non-null   float64
 4   value   2535 non-null   int64  
 5   volume  2535 non-null   int64  
dtypes: float64(4), int64(2)
memory usage: 138.6 KB


In [161]:
#прописываем скользящие средние и в одну таблицу собираем
short_window = data.close.rolling(short_wind, min_periods = 0).mean()
short_window = short_window.rename('Short') 
long_window =data.close.rolling(90, min_periods = 0).mean()
df_window = pd.concat([short_window, long_window], axis = 1)
df_window = df_window.rename(columns={'close': 'Long'})
df_window

,Short,Long
begin,,
2018-01-03 09:00:00,679.800000,679.800000
2018-01-03 10:00:00,681.050000,681.050000
2018-01-03 11:00:00,684.333333,684.333333
2018-01-03 12:00:00,686.600000,686.600000
2018-01-03 13:00:00,687.700000,687.700000
...,...,...
2018-12-29 14:00:00,1114.816667,1110.932222
2018-12-29 15:00:00,1116.096667,1111.194444
2018-12-29 16:00:00,1117.606667,1111.393333


In [162]:
#создаем датафрейм с нулями, сохраняя только стоимость привязанную ко времени. По сути это база для робота
df = pd.DataFrame({'signal': 0, 'num_shares': 0, 'share_price': data.open, 'share_value': 0, 'cash': 0})
df["cash"].iloc[0] = money
df["num_shares"].iloc[0] = 0
df = df.reset_index().rename(columns={'Date': 'date'})
df

,begin,signal,num_shares,share_price,share_value,cash
0,2018-01-03 09:00:00,0,0,679.8,0,100000
1,2018-01-03 10:00:00,0,0,679.3,0,0
2,2018-01-03 11:00:00,0,0,683.7,0,0
3,2018-01-03 12:00:00,0,0,692.3,0,0
4,2018-01-03 13:00:00,0,0,694.5,0,0
...,...,...,...,...,...,...
2530,2018-12-29 14:00:00,0,0,1128.6,0,0
2531,2018-12-29 15:00:00,0,0,1129.6,0,0
2532,2018-12-29 16:00:00,0,0,1131.7,0,0
2533,2018-12-29 17:00:00,0,0,1132.1,0,0


In [163]:
df_window["Option"] = df_window.apply(lambda x: "Long" if x["Short"] < x["Long"] else ("Short" if x["Short"] > x["Long"] else 0), axis=1)

for i in range(1, len(df_window)):
    if df_window.Option.iloc[i] == "Long" and df_window.Option.iloc[i-1] == "Short":
        df.signal.iloc[i] = "sig_sale"
    elif df_window.Option.iloc[i] == "Short" and df_window.Option.iloc[i-1] == "Long":
        df.signal.iloc[i] = "sig_buy"

In [164]:
# Создание пустого DataFrame
df_result = pd.DataFrame(columns=["begin", "signal", "num_shares", "share_price", "share_value", "cash"])

def update_dataframe(df_result, begin, signal, num_shares, share_price, share_value, cash):
    df_result.loc[len(df_result.index)] = [begin, signal, num_shares, share_price, share_value, cash]
    
def process_signals(df_result, data, df, stop_loss):
    list1 = [0]
    stloss = [0]
    bought_price = 0
    sale_signal = False  # Флаг для отслеживания сигнала на продажу
    
    for i in range(len(df)):
        if df.signal[i] == "sig_buy":
            list1.pop(0)
            bought_price = data.open[i+1]
            update_dataframe(df_result, df.begin[i], "sig_buy", 0, data.open[i], 0, 0)
            update_dataframe(df_result, df.begin[i+1], "buy", 0, data.open[i+1], 0, 0)
            stl = bought_price * (1 - stop_loss)
            stloss[0] = stl
            sale_signal = False  # Сброс флага при покупке
        
        elif df.signal[i] == "sig_sale" and len(list1) == 0:
            list1.append(0)
            update_dataframe(df_result, df.begin[i], "sig_sale", 0, data.open[i], 0, 0)
            update_dataframe(df_result, df.begin[i+1], "sale", 0, data.open[i+1], 0, 0)
            sale_signal = True  # Установка флага при сигнале на продажу
        
        elif bought_price > 0 and not sale_signal and (data.open[i] / bought_price) <= (1 - stop_loss):
            list1.append(0)
            update_dataframe(df_result, df.begin[i], "stop-loss", 0, data.open[i], 0, 0)
            stloss[0] = 0
            bought_price = 0

            
# Вызов функции для обработки сигналов
process_signals(df_result, data, df, stop_loss)

In [165]:
#дальше комментариев не будет, я не успеваю(
df_result.cash[0] = money

for i in range(1, len(df_result)):
    if df_result['signal'][i] == 'sig_buy':
        df_result['num_shares'][i] = 0
        df_result['share_value'][i] = df_result['num_shares'][i] * df_result['share_price'][i]
        df_result['cash'][i] = df_result['cash'][i-1]
    
    elif df_result['signal'][i] == 'buy':
        df_result['num_shares'][i] = df_result['cash'][i-1] // df_result['share_price'][i]
        df_result['share_value'][i] = df_result['num_shares'][i] * df_result['share_price'][i]
        df_result['cash'][i] = df_result['cash'][i-1] - df_result['share_value'][i]
    
    elif df_result['signal'][i] == 'sig_sale':
        df_result['num_shares'][i] = df_result['num_shares'][i-1]
        df_result['share_value'][i] = df_result['num_shares'][i] * df_result['share_price'][i]
        df_result['cash'][i] = df_result['cash'][i-1]
    
    elif df_result['signal'][i] == 'sale':
        df_result['num_shares'][i] = df_result['num_shares'][i-1]
        df_result['share_value'][i] = 0
        df_result['cash'][i] = (df_result['num_shares'][i-1] * df_result['share_price'][i]) + df_result['cash'][i-1]
    
    elif df_result['signal'][i] == 'stop-loss':
        df_result['num_shares'][i] = df_result['num_shares'][i-1]
        df_result['cash'][i] = (df_result['num_shares'][i-1] * df_result['share_price'][i]) + df_result['cash'][i-1]
        df_result['share_value'][i] = (df_result['num_shares'][i-1] * df_result['share_price'][i])
        
df_result['cash'] = round(df_result['cash'], 2)
df_result['share_price'] = round(df_result['share_price'], 2)
df_result['share_value'] = round(df_result['share_value'], 2)
df_result

,begin,signal,num_shares,share_price,share_value,cash
0,2018-01-18 16:00:00,sig_buy,0,694.3,0.0,100000.0
1,2018-01-18 17:00:00,buy,144,693.5,99864.0,136.0
2,2018-02-12 16:00:00,sig_sale,144,729.7,105076.8,136.0
3,2018-02-12 17:00:00,sale,144,720.4,0.0,103873.6
4,2018-02-16 09:00:00,sig_buy,0,744.9,0.0,103873.6
...,...,...,...,...,...,...
59,2018-12-18 09:00:00,buy,90,1117.0,100530.0,1038.2
60,2018-12-25 09:00:00,sig_sale,90,1105.0,99450.0,1038.2
61,2018-12-25 10:00:00,sale,90,1104.1,0.0,100407.2
62,2018-12-29 10:00:00,sig_buy,0,1126.8,0.0,100407.2


In [166]:
sd_list = df_result[(df_result == 'sale').any(axis=1)].index.values.tolist()
ds_list = df_result[(df_result == 'stop-loss').any(axis=1)].index.values.tolist()
ob_list = sd_list + ds_list


value = df_result.cash[max(ob_list)]
perc_grow = ((df_result.cash[max(ob_list)] / df_result.cash[0]) * 100) - 100

print("Итоговая стоимость на момент последней продажи акций", round(value, 2), "денежные единицы")
print("Процент прироста/падения исходных вложений:", round(perc_grow, 2), "%") 

Итоговая стоимость на момент последней продажи акций 100407.2 денежные единицы
Процент прироста/падения исходных вложений: 0.41 %


In [167]:
inc = data.close > data.open
dec = data.open > data.close
w = 12*60*60*1000 

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"

v = figure(x_axis_type="datetime", tools=TOOLS, width=1000, title = "MSFT Candlestick")
v.xaxis.major_label_orientation = pi/4
v.grid.grid_line_alpha=0.3

v.segment(data.index, data.high, data.index, data.low, color="black")
v.vbar(data.index[inc], w, data.open[inc], data.close[inc], fill_color="green", line_color="green")
v.vbar(data.index[dec], w, data.open[dec], data.close[dec], fill_color="#F36223", line_color="#F36223")

v.line(short_window.index, short_window, legend_label="Скользящая короткая", color="mediumpurple", line_width=2)
v.line(long_window.index, long_window, legend_label="Скользящая длинная", color="hotpink", line_width=2)

for i in range(0, len(df_result)):
    if df_result.signal[i] == "buy":
        v.triangle(df_result.begin[i], [df_result.share_price[i]-3], size=10, color="green", alpha=1)
    if df_result.signal[i] == "sale":
        v.inverted_triangle(df_result.begin[i], [df_result.share_price[i]+3], size=10, color="red", alpha=1)
    if df_result.signal[i] == "stop-loss":
        v.inverted_triangle(df_result.begin[i], [df_result.share_price[i]+3], size=10, color="black", alpha=1)

show(v)

In [168]:
newdf = pd.DataFrame(df_result.begin)
newdf["signal"] = df_result.signal
newdf["sig"] = 0
newdf = newdf.set_index("begin")
newdf2 = pd.DataFrame(columns = ["begin", "signal", "sig"])
for i in range(0, len(newdf)):
    if newdf["signal"][i] == "sig_buy":
        newdf2.loc[len(newdf2)] = [newdf.index[i], "sig_buy", 0]
        newdf2.loc[len(newdf2)] = [newdf.index[i], "buy", 1]
    if newdf["signal"][i] == "sig_sale":
        newdf2.loc[len(newdf2)] = [newdf.index[i], "sig_sale", 1]
        newdf2.loc[len(newdf2)] = [newdf.index[i], "sale", 0]
    if newdf["signal"][i] == "stop-loss":
        newdf2.loc[len(newdf2)] = [newdf.index[i], "stop-loss", 1]
        newdf2.loc[len(newdf2)] = [newdf.index[i], "stop-loss", 0]
newdf2['date'] = pd.to_datetime(newdf2["begin"])

z = figure()
z.line(newdf2.date, newdf2.sig)
show(z)